# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [1]:
# import numpy and pandas
import numpy as np
import pandas as pd


# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents.

In [2]:
# Your code here:
chicago = pd.read_csv('Current_Employee_Names__Salaries__and_Position_Titles.csv')

Examine the `salaries` dataset using the `head` function below.

In [3]:
# Your code here:

chicago.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [4]:
# Your code here:

nan_counts = chicago.isnull().sum(axis = 0)
nan_counts

Name                     0
Job Titles               0
Department               0
Full or Part-Time        0
Salary or Hourly         0
Typical Hours        25161
Annual Salary         8022
Hourly Rate          25161
dtype: int64

Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [5]:
# Your code here:
chicago['Salary or Hourly'].value_counts()

Salary    25161
Hourly     8022
Name: Salary or Hourly, dtype: int64

What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [6]:
# Your code here:

department_table1 = chicago.pivot_table(index= ['Department'],  aggfunc = {'Name':'count'})
department_table = department_table1.rename(columns={"Name": "Number of Employees"})
department_table

,Number of Employees
Department,
ADMIN HEARNG,39
ANIMAL CONTRL,81
AVIATION,1629
BOARD OF ELECTION,107
BOARD OF ETHICS,8
BUDGET & MGMT,46
BUILDINGS,269
BUSINESS AFFAIRS,171
CITY CLERK,84


# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [7]:
# Your code here:

#create list of hourly rates

hourly_list_nans = chicago['Hourly Rate'].tolist()
hourly_list_nans

[nan,
 nan,
 nan,
 nan,
 19.86,
 nan,
 46.1,
 35.6,
 nan,
 nan,
 2.65,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 17.68,
 46.1,
 nan,
 21.43,
 nan,
 nan,
 35.6,
 nan,
 25.1,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 20.0,
 nan,
 nan,
 36.21,
 nan,
 nan,
 35.6,
 45.07,
 nan,
 44.88,
 nan,
 nan,
 nan,
 nan,
 nan,
 46.1,
 nan,
 nan,
 nan,
 48.25,
 nan,
 nan,
 nan,
 36.22,
 40.2,
 nan,
 nan,
 nan,
 nan,
 36.13,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 40.2,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 32.04,
 nan,
 nan,
 28.48,
 nan,
 nan,
 nan,
 nan,
 17.68,
 47.5,
 nan,
 nan,
 46.1,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 36.21,
 nan,
 40.2,
 nan,
 nan,
 36.21,
 32.25,
 nan,
 nan,
 46.35,
 21.2,
 nan,
 nan,
 nan,
 35.6,
 nan,
 2.65,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 40.2,
 nan,
 nan,
 36.21,
 nan,
 nan,
 nan,
 nan,
 28.48,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 20.77,
 nan,
 13.15,
 nan,
 21.98,
 nan,
 nan,
 35.6,
 nan,
 nan,
 nan,
 na

In [8]:
#clean list from nans

hourly_list = [x for x in hourly_list_nans if str(x) != 'nan']
hourly_list

[19.86,
 46.1,
 35.6,
 2.65,
 17.68,
 46.1,
 21.43,
 35.6,
 25.1,
 20.0,
 36.21,
 35.6,
 45.07,
 44.88,
 46.1,
 48.25,
 36.22,
 40.2,
 36.13,
 40.2,
 32.04,
 28.48,
 17.68,
 47.5,
 46.1,
 36.21,
 40.2,
 36.21,
 32.25,
 46.35,
 21.2,
 35.6,
 2.65,
 40.2,
 36.21,
 28.48,
 20.77,
 13.15,
 21.98,
 35.6,
 15.94,
 20.12,
 40.2,
 35.6,
 40.2,
 19.86,
 12.49,
 35.6,
 36.21,
 36.21,
 35.6,
 36.21,
 35.6,
 32.04,
 36.21,
 35.6,
 36.21,
 35.6,
 34.57,
 16.17,
 13.94,
 35.6,
 35.6,
 35.6,
 8.25,
 47.44,
 19.86,
 36.21,
 35.6,
 19.86,
 2.65,
 46.35,
 35.6,
 32.59,
 43.03,
 36.22,
 46.35,
 8.25,
 40.2,
 36.21,
 15.22,
 28.14,
 40.2,
 45.07,
 19.86,
 35.6,
 40.2,
 19.38,
 48.25,
 18.52,
 36.21,
 44.25,
 2.65,
 13.94,
 21.3,
 36.21,
 49.1,
 18.52,
 15.65,
 19.38,
 40.2,
 47.44,
 36.21,
 47.44,
 40.2,
 15.0,
 44.25,
 47.44,
 32.04,
 21.98,
 19.86,
 18.15,
 22.36,
 36.21,
 18.52,
 25.1,
 35.6,
 46.35,
 35.6,
 35.6,
 35.57,
 19.86,
 14.54,
 35.6,
 36.21,
 36.21,
 22.12,
 36.21,
 45.07,
 35.6,
 49.1,
 35.

In [9]:
# H0 : hourly rate mean = 30

#confidence_level = 0.95
#significance_level = 1 - 0.95 = 0.05 
#for a two sided sided confidence interval we will compare to 0.05/2 = 0.025


from scipy import stats

test = stats.ttest_1samp(hourly_list, 30)
test

#here we can see that the p-value is much smaller than the significance level which indicates we should reject the H0

Ttest_1sampResult(statistic=20.6198057854942, pvalue=4.3230240486229894e-92)

We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

In [10]:
# Your code here:

#get a list of all Police Salaries

police_frame = chicago[(chicago['Salary or Hourly'] == 'Salary') & (chicago['Department'] == 'POLICE')]
police_list = police_frame['Annual Salary'].tolist()
police_list

[101442.0,
 94122.0,
 93354.0,
 84054.0,
 87006.0,
 84054.0,
 90024.0,
 48078.0,
 76266.0,
 48078.0,
 80016.0,
 72510.0,
 68616.0,
 90024.0,
 48078.0,
 76266.0,
 48078.0,
 72510.0,
 97440.0,
 72510.0,
 87006.0,
 90024.0,
 84054.0,
 87006.0,
 90024.0,
 72510.0,
 87006.0,
 43800.0,
 76266.0,
 84054.0,
 84054.0,
 90024.0,
 100980.0,
 90024.0,
 90024.0,
 90024.0,
 87006.0,
 93354.0,
 93354.0,
 72510.0,
 93354.0,
 50676.0,
 87006.0,
 72510.0,
 90024.0,
 72510.0,
 84054.0,
 94524.0,
 48078.0,
 90024.0,
 100980.0,
 96060.0,
 100776.0,
 87006.0,
 40392.0,
 48078.0,
 68616.0,
 100980.0,
 93354.0,
 48078.0,
 48078.0,
 84054.0,
 76266.0,
 84054.0,
 97440.0,
 90024.0,
 94524.0,
 84054.0,
 111474.0,
 90024.0,
 50676.0,
 96060.0,
 91338.0,
 96060.0,
 111474.0,
 93354.0,
 87006.0,
 96060.0,
 103932.0,
 94524.0,
 111474.0,
 87006.0,
 84054.0,
 84054.0,
 104628.0,
 90024.0,
 48078.0,
 87006.0,
 76266.0,
 90024.0,
 80016.0,
 88296.0,
 96060.0,
 87006.0,
 90024.0,
 84054.0,
 84054.0,
 103932.0,
 100980.0

In [11]:
# H0 : mean_police_salaries > 86000
test2 = stats.ttest_1samp(police_list, 86000)
test2

#confidence_level = 0.95
#significance_level = 1 - 0.95 = 0.05 
#for a two sided sided confidence interval we will compare to 0.05/2 = 0.025

#"If our hypothesis is greater than, then to reject, the test statistic must also be positive."

#statistic is positive but p-value < 0.025 so we reject H0

Ttest_1sampResult(statistic=3.081997005712994, pvalue=0.0020603403550965137)

Using the `crosstab` function, find the department that has the most hourly workers. 

In [12]:
# Your code here:

#read about 'crosstab': https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.crosstab.html

dep_most_hourly =pd.crosstab(chicago['Department'], chicago['Salary or Hourly']).sort_values(by='Hourly', ascending= False)
dep_most_hourly


Salary or Hourly,Hourly,Salary
Department,,
STREETS & SAN,1862,336
WATER MGMNT,1513,366
OEMC,1273,829
AVIATION,1082,547
GENERAL SERVICES,765,215
TRANSPORTN,725,415
PUBLIC LIBRARY,299,716
FAMILY & SUPPORT,287,328
CITY COUNCIL,64,347


In [13]:
department = dep_most_hourly.loc[dep_most_hourly['Hourly'].idxmax()]
department

Salary or Hourly
Hourly    1862
Salary     336
Name: STREETS & SAN, dtype: int64

The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [14]:
# Your code here:

street_sanitation_frame = chicago[(chicago['Salary or Hourly'] == 'Hourly') & (chicago['Department'] == 'STREETS & SAN')]
street_sanitation_list = street_sanitation_frame['Hourly Rate'].tolist()
# H0 : mean < 35

# t student distribution
# one sided -> 0.5
alpha = 0.5
test3 = stats.ttest_1samp(street_sanitation_list, 35)

if test3[1] < alpha :
    print('We reject the null hypothesis with a p-value of',test3[1])
else :
    print('We accept the null hypothesis with a p-value of',test3[1])



We reject the null hypothesis with a p-value of 3.3378530564707717e-21


# Challenge 3: To practice - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [19]:
from scipy.stats import t

# number of rows - 1 for degrees of freedom
dfreedom = len(hourly_list)-1

# mean of the sample for the location parameter
location = np.mean(hourly_list)

# the standard error for the scale
n = len(hourly_list)
scale = np.std(hourly_list) / np.sqrt(n)

In [21]:
tstudent = t(dfreedom,location,scale)

confidence_interval = tstudent.interval(0.95)

print("The confidence interval for the hourly wage with a 0.95 confidence interval is", confidence_interval)

The confidence interval for the hourly wage with a 0.95 confidence interval is (32.52347486867093, 33.053640563889545)


Now construct the 95% confidence interval for all salaried employeed in the police in the cell below.

In [23]:
# number of rows - 1 for degrees of freedom
dfreedom2 = len(police_list)-1

# mean of the sample for the location parameter
location2 = np.mean(police_list)

# the standard error for the scale
n2 = len(police_list)
scale2 = np.std(police_list) / np.sqrt(n2)

tstudent2 = t(dfreedom2,location2,scale2)

confidence_interval2 = tstudent2.interval(0.95)

print("The confidence interval for the hourly wage with a 0.95 confidence interval is", confidence_interval2)

The confidence interval for the hourly wage with a 0.95 confidence interval is (86177.06785530433, 86795.76115096245)


# Bonus Challenge - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [ ]:
# Your code here:

